# TraceMem + LangChain Example

This notebook demonstrates importing LangChain conversation traces into TraceMem
and querying them using the retrieval API.

## Prerequisites

- Neo4j running locally (`docker compose up -d neo4j`)
- `langchain-core` installed (`uv add langchain-core`)

## 1. Setup

Create a mock embedder (no OpenAI key needed), configure TraceMem with
a temp LanceDB path to avoid conflicts with existing data, and connect.

In [4]:
import random
import tempfile
from pathlib import Path

from tracemem_core import TraceMem, TraceMemConfig, RetrievalConfig, Embedder
from tracemem_core.adapters.langchain import LangChainAdapter


class MockEmbedder(Embedder):
    """Deterministic embedder for demo purposes (no API key needed)."""

    @property
    def dimensions(self) -> int:
        return 256

    async def embed(self, text: str) -> list[float]:
        rng = random.Random(text)
        vec = [rng.gauss(0, 1) for _ in range(self.dimensions)]
        norm = sum(x * x for x in vec) ** 0.5
        return [x / norm for x in vec]

    async def embed_batch(self, texts: list[str]) -> list[list[float]]:
        return [await self.embed(t) for t in texts]


# Use a temp directory for LanceDB so we don't conflict with existing data
_tmpdir = tempfile.mkdtemp(prefix="tracemem_example_")

config = TraceMemConfig(
    mode="global",
    embedding_dimensions=256,
    lancedb_path=Path(_tmpdir) / "lancedb",
    reranker="rrf",
    retrieval=RetrievalConfig(limit=5, include_context=True),
)

adapter = LangChainAdapter()
embedder = MockEmbedder()

# Open a single TraceMem connection for the whole notebook
tm = TraceMem(config=config, embedder=embedder)
await tm.__aenter__()

print(f"Connected. LanceDB at: {_tmpdir}")

Connected. LanceDB at: /var/folders/lf/j9dpx4lx3bl0x2tgdkr0pn8c0000gn/T/tracemem_example_pni8ts5k


## 2. Import a LangChain Conversation

In [5]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

langchain_messages = [
    HumanMessage(content="Read the auth.py file and explain the login flow"),
    AIMessage(
        content="I'll read the file for you.",
        tool_calls=[
            {"id": "call_1", "name": "read_file", "args": {"file_path": "/src/auth.py"}}
        ],
    ),
    ToolMessage(
        content="def login(user, password):\n    token = create_jwt(user)\n    return token",
        tool_call_id="call_1",
    ),
    AIMessage(content="The login flow creates a JWT token from the user credentials and returns it."),
    HumanMessage(content="Now add rate limiting to the login endpoint"),
    AIMessage(
        content="I'll update auth.py to add rate limiting.",
        tool_calls=[
            {"id": "call_2", "name": "edit_file", "args": {"file_path": "/src/auth.py"}}
        ],
    ),
    ToolMessage(content="File updated successfully", tool_call_id="call_2"),
    AIMessage(content="I've added a rate limiter decorator to the login function."),
]

messages = adapter.convert(langchain_messages)
result = await tm.import_trace("conv-auth-demo", messages)
print(f"Imported nodes: {list(result.keys())}")

Imported nodes: ['user_text', 'agent_text', 'resource_file:///src/auth.py', 'resource_version_file:///src/auth.py']


## 3. Import a Second Conversation

Import another conversation so we have data to search across.

In [6]:
langchain_messages_2 = [
    HumanMessage(content="How does the database connection pool work?"),
    AIMessage(
        content="Let me check the database module.",
        tool_calls=[
            {"id": "call_3", "name": "read_file", "args": {"file_path": "/src/database.py"}}
        ],
    ),
    ToolMessage(content="pool = create_pool(max_connections=10)", tool_call_id="call_3"),
    AIMessage(content="The database uses a connection pool with max 10 connections."),
    HumanMessage(content="Fix the authentication token expiry bug"),
    AIMessage(
        content="I'll look at the auth module.",
        tool_calls=[
            {"id": "call_4", "name": "read_file", "args": {"file_path": "/src/auth.py"}}
        ],
    ),
    ToolMessage(
        content="def create_jwt(user):\n    return jwt.encode({'exp': time() + 3600}, SECRET)",
        tool_call_id="call_4",
    ),
    AIMessage(content="Found the bug - the token expiry was using time() instead of datetime.utcnow()."),
]

messages_2 = adapter.convert(langchain_messages_2)
result = await tm.import_trace("conv-db-auth-demo", messages_2)
print(f"Imported nodes: {list(result.keys())}")

Imported nodes: ['user_text', 'agent_text', 'resource_file:///src/database.py', 'resource_version_file:///src/database.py', 'resource_version_file:///src/auth.py']


## 4. Search Past Interactions

Use `tm.search()` to find relevant past conversations.

In [7]:
results = await tm.search("authentication issues")

for r in results:
    print(r)
    print()

Result(e1827c35, score=0.032, conv=conv-db-auth-demo, text='Fix the authentication token expiry bug', context=yes)

Result(e7841365, score=0.016, conv=conv-db-auth-demo, text='How does the database connection pool work?', context=yes)

Result(5cdb8835, score=0.016, conv=conv-auth-demo, text='Read the auth.py file and explain the login flow', context=yes)

Result(7bee1af5, score=0.016, conv=conv-auth-demo, text='Now add rate limiting to the login endpoint', context=yes)



## 5. Query File History

Find all conversations that touched a specific file.

In [8]:
refs = await tm.get_conversations_for_resource(
    "file:///src/auth.py",
    config=RetrievalConfig(limit=10, sort_by="created_at", sort_order="desc"),
)

for ref in refs:
    print(ref)
    

ConvRef(e1827c35, conv=conv-db-auth-demo, user='Fix the authentication token expiry bug')
ConvRef(e7841365, conv=conv-db-auth-demo, user='How does the database connection pool work?')
ConvRef(7bee1af5, conv=conv-auth-demo, user='Now add rate limiting to the login endpoint')
ConvRef(5cdb8835, conv=conv-auth-demo, user='Read the auth.py file and explain the login flow')
ConvRef(5cdb8835, conv=conv-auth-demo, user='Read the auth.py file and explain the login flow')


## 6. Get Full Trajectory

Expand a search result into its full trajectory (user message + all agent responses + tool uses).

In [9]:
results = await tm.search("login flow", config=RetrievalConfig(limit=1))

if results:
    trajectory = await tm.get_trajectory(results[0].node_id)
    print(trajectory)
else:
    print("No results found")

Trajectory(3 steps):
  Step(7bee1af5 UserText: 'Now add rate limiting to the login endpoint')
  Step(b109d83f AgentText: "I'll update auth.py to add rate limiting." tools=1)
  Step(33920720 AgentText: "I've added a rate limiter decorator to the login function.")


## 7. Reranker Selection

Create a second TraceMem instance with a different reranker to compare.

In [10]:
linear_config = TraceMemConfig(
    mode="global",
    embedding_dimensions=256,
    lancedb_path=Path(_tmpdir) / "lancedb",
    reranker="linear",
)

async with TraceMem(config=linear_config, embedder=embedder) as tm_linear:
    results = await tm_linear.search("authentication")
    print(f"Results with linear reranker: {len(results)}")
    for r in results:
        print(f"  {r.text[:60]}  (score={r.score:.3f})")

Results with linear reranker: 4
  Fix the authentication token expiry bug  (score=1.000)
  Now add rate limiting to the login endpoint  (score=0.374)
  Read the auth.py file and explain the login flow  (score=0.021)
  How does the database connection pool work?  (score=0.000)


## 8. Cleanup

Close the TraceMem connection and remove temp files.

In [11]:
import shutil

await tm.__aexit__(None, None, None)
shutil.rmtree(_tmpdir, ignore_errors=True)
print("Cleaned up")

Cleaned up
